In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import gpxpy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from gps_analysis import geodesy, files, splits, garmin

In [5]:
api = garmin.login("imogen@gtsoftware.co.uk")

In [6]:
activities = garmin.get_activities(0, 10, activityType='rowing')

In [7]:
activity_data, errors = garmin.load_fit_activities(activities.activityId)

  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
locations=None
xlpath='garmin_data.xlsx'
api=None
max_workers=4
show_progress=True

In [19]:
actid, data = list(activity_data.items())[0]
positions = data
distances = splits._STANDARD_DISTANCES

distance = 2

In [66]:
splits.find_all_best_times(positions, cols=cols)

time                     split  \
length distance                                                       
250m   5.135    0 days 00:00:44.729112992 0 days 00:01:29.458225984   
       9.452    0 days 00:00:45.074978204 0 days 00:01:30.149956408   
       5.928    0 days 00:00:45.513612951 0 days 00:01:31.027225902   
       5.618    0 days 00:00:45.909090909 0 days 00:01:31.818181818   
       10.338   0 days 00:00:46.058795525 0 days 00:01:32.117591050   
...                                   ...                       ...   
3km    9.346    0 days 00:14:54.744158416 0 days 00:02:29.124026402   
5km    1.623    0 days 00:24:36.739336493 0 days 00:02:27.673933649   
       9.330    0 days 00:25:47.205305652 0 days 00:02:34.720530565   
7km    1.623    0 days 00:35:20.311230964 0 days 00:02:31.450802211   
10km   0.927    0 days 00:51:22.681536555 0 days 00:02:34.134076827   

                 heart_rate  cadence  bearing  
length distance                                
250m   5.135          153.5     37.2     17.5  
       9.452          153.9     34.6     17.4  
       5.928          175.5     35.0     16.9  
       5.618          173.6     35.0     17.2  
       10.338         173.8     35.2     17.1  
...                     ...      ...      ...  
3km    9.346          143.5     19.6    -49.5  
5km    1.623          140.4     20.1    -40.0  
       9.330          141.5     17.8    -92.8  
7km    1.623          142.5     20.0    -56.4  
10km   0.927          139.5     19.5    -46.2  

[101 rows x 5 columns]

In [30]:
cols = ['heart_rate', 'cadence', 'bearing']

In [63]:
splits.find_best_times(positions, 2, cols)

,time,split,heart_rate,cadence,bearing
distance,,,,,
1.203,0 days 00:07:59.494897959,0 days 00:01:59.873724489,140.1,22.9,16.6
4.886,0 days 00:08:05.802218115,0 days 00:02:01.450554528,150.7,24.1,14.3
11.290,0 days 00:08:49.042867701,0 days 00:02:12.260716925,157.6,23.9,-163.7
7.077,0 days 00:08:58.721068249,0 days 00:02:14.680267062,161.3,25.0,-163.9
9.163,0 days 00:10:18.754045307,0 days 00:02:34.688511326,138.1,16.5,5.0
13.296,0 days 00:11:40.938028169,0 days 00:02:55.234507042,131.5,13.8,-118.1


In [62]:
avg_col_vals = pd.DataFrame([
    dist_cols[d:d + distance][cols].mean(0)
    for d in distances.loc[best]
], columns=cols).round(1)
avg_col_vals.index = best
avg_col_vals

,heart_rate,cadence,bearing
103,140.1,22.9,16.6
521,150.7,24.1,14.3
1070,157.6,23.9,-163.7
731,161.3,25.0,-163.9
886,138.1,16.5,5.0
1194,131.5,13.8,-118.1


In [20]:
    total_distance = positions.distance.iloc[-1]
    time_elapsed = positions.timeElapsed.dt.total_seconds()
    sel = positions.distance + distance < total_distance
    distances = positions.distance[sel]
    end_distances = distances + distance

    dist_elapsed = np.interp(
        end_distances, positions.distance, time_elapsed    
    )
    other_cols = {
        c: np.interp(end_distances, positions.distance, positions[c])
        for c in cols
    }

    dist_times = dist_elapsed - time_elapsed[sel] 
    best_ordering = dist_times.argsort().values

    best = []
    unblocked = np.ones_like(best_ordering, dtype=bool)
    while unblocked.any():
        next_best = best_ordering[unblocked[best_ordering]][0]
        best.append(next_best)
        i0 = end_distances.searchsorted(distances[next_best])
        i1 = distances.searchsorted(end_distances[next_best])
        unblocked[i0:i1] = False
    
    best_times = pd.to_timedelta(dist_times[best], 'S')
    best_timesplits = pd.DataFrame.from_dict({
        'time': best_times,
        'split': best_times / distance / 2
    })
    best_timesplits.index = distances[best].round(3)

In [28]:
best_timesplits

,time,split
distance,,
1.203,0 days 00:07:59.494897959,0 days 00:01:59.873724489
4.886,0 days 00:08:05.802218115,0 days 00:02:01.450554528
11.290,0 days 00:08:49.042867701,0 days 00:02:12.260716925
7.077,0 days 00:08:58.721068249,0 days 00:02:14.680267062
9.163,0 days 00:10:18.754045307,0 days 00:02:34.688511326
13.296,0 days 00:11:40.938028169,0 days 00:02:55.234507042


In [27]:
distances[best]

103      1.20293
521      4.88595
1070    11.28964
731      7.07715
886      9.16348
1194    13.29626
Name: distance, dtype: float64

In [26]:
best

[103, 521, 1070, 731, 886, 1194]

In [21]:
positions

,time,distance,unknown_88,heart_rate,cadence,fractional_cadence,position_lat,position_long,enhanced_speed,speed,unknown_87,latitude,longitude,timeElapsed,distanceDelta,bearing_r,bearing
0,2022-03-12 10:15:50,0.00232,300,86,19,0.0,625141790.0,3413670.0,0.466,0.466,147.0,52.398780,0.286131,0 days 00:00:00,0.00224,0.996115,57.073167
1,2022-03-12 10:15:52,0.00456,300,83,19,0.0,625141920.0,3413999.0,0.630,0.630,199.0,52.398790,0.286158,0 days 00:00:02,0.00257,1.029379,58.979096
2,2022-03-12 10:15:55,0.00713,300,80,16,0.0,625142062.0,3414386.0,0.693,0.693,260.0,52.398802,0.286191,0 days 00:00:05,0.00196,1.137796,65.190936
3,2022-03-12 10:15:58,0.00909,300,79,18,0.0,625142150.0,3414698.0,0.753,0.753,251.0,52.398810,0.286217,0 days 00:00:08,0.00204,0.923199,52.895397
4,2022-03-12 10:16:00,0.01113,300,76,18,0.0,625142282.0,3414984.0,0.868,0.868,289.0,52.398821,0.286241,0 days 00:00:10,0.00527,0.861336,49.350914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409,2022-03-12 11:41:11,15.32940,300,104,0,0.0,625140922.0,3412072.0,0.000,0.000,NaN,52.398707,0.285997,0 days 01:25:21,0.00249,-2.795049,-160.144483
1410,2022-03-12 11:41:17,15.33189,300,107,0,0.0,625140677.0,3411927.0,0.000,0.000,NaN,52.398686,0.285984,0 days 01:25:27,0.00281,-2.515748,-144.141764
1411,2022-03-12 11:41:28,15.33470,300,110,0,0.0,625140444.0,3411651.0,0.000,0.000,NaN,52.398667,0.285961,0 days 01:25:38,0.00009,-1.887541,-108.148158
1412,2022-03-12 11:41:30,15.33479,300,110,0,0.0,625140441.0,3411636.0,0.000,0.000,NaN,52.398666,0.285960,0 days 01:25:40,0.00333,2.509282,143.771294


In [13]:

    activity_td = activities[
        ['activityId', 'startTimeLocal', 'distance']
    ].sort_values(by='startTimeLocal', ascending=False)
    activity_td.columns = ['activityId', 'startTime', 'totalDistance']

    # reorder activity_data
    activity_data = {i: activity_data[i] for i in activity_td.activityId}

    activity_td.totalDistance = (activity_td.totalDistance/1000).round(1)

    activity_best_times = pd.concat({
        actid: splits.find_all_best_times(data)
        for actid, data in activity_data.items() if not data.empty
    }, 
        names = ('activityId', 'length', 'distance')
    )

    best_times = activity_best_times.reset_index().join(
        activity_td.set_index('activityId'), on='activityId').set_index(
        ['activityId', 'startTime', 'totalDistance', 'length', 'distance']
    )

    sheet_names = pd.Series(
        activities.startTimeLocal.str.split(" ", expand=True)[0].str.cat(
            activities.activityId.astype(str), sep="_"
        ).values,
        index = activities.activityId
    )
    location_timings = {
        actid: splits.get_location_timings(data, locations)
        for actid, data in activity_data.items() if not data.empty
    }

/mnt/c/Users/matthew/Source/gps_analysis/gps_analysis/splits.py:90: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series([])
/mnt/c/Users/matthew/Source/gps_analysis/gps_analysis/splits.py:90: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series([])
/mnt/c/Users/matthew/Source/gps_analysis/gps_analysis/splits.py:90: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series([])
/mnt/c/Users/matthew/Source/gps_analysis/gps_analysis/splits.py:97: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  crossing_weig

In [54]:
activity_data, errors = garmin.load_activities(activities.activityId)

  0%|          | 0/10 [00:00<?, ?it/s]

In [55]:
garmin.activity_data_to_excel(activities, activity_data)

In [ ]:
locations = {
    'start': [51.461398172524824, -0.9455592979289794],
    '500m': [51.46460471321316, -0.9404994529495201],
    '1km': [51.467811036115336, -0.9354388968526993],
    '1.5km': [51.47101714117572, -0.930377629505215], 
    'finish': [51.47420259934778, -0.9253479107461384],
}
locations = pd.DataFrame.from_dict({
    k: {
        'latitude': v[0], 
        'longitude': v[1],
        'bearing': geodesy.estimate_bearing(
            positions, geodesy.LatLon(*v), 0.05)
    } 
    for k, v in locations.items()
},
    orient='index')
locations.index.name='location'
locations.bearing = geodesy.bearing(
    locations.loc['start'],
    locations.loc['finish']
)

In [ ]:
locations.to_csv("data/cav_locations.tsv", sep='\t')

In [ ]:
actid = 8026393665
actid = 2366290503

positions = garmin.load_activity(actid, api=api)

In [ ]:
garmin.login('imogen@gtsoftware.co.uk')
actid = 2366290503
data = garmin.load_fit_activity(actid)

In [ ]:
data.time.diff(-1)

In [ ]:
best_splits = splits.find_all_best_times(data)

In [ ]:
best_splits.loc["2km"]

In [ ]:
def interval_average_value(locations, label, start_dist, dist):
    interval = [start_dist, start_dist + dist]
    i0, i1 = locations.distance.searchsorted(interval)
    val = locations.loc[i0:i1, label]
    times = locations.timeElapsed.dt.total_seconds().copy()
    times.loc[[i0, i1 + 1]] = np.interp(interval, locations.distance, times)
    weights = -times.loc[i0:i1+1].diff(-1)[:-1]
    return np.average(val, weights=weights)

interval_average_value(data, 'cadence', 12.041, 2)

In [ ]:
interval_average_value(data, 'cadence', 3.4, 2)

In [ ]:
data.distance.diff(-1)

In [ ]:
plt.figure()
plt.plot(data.distance, data.cadence)

In [ ]:
distance = 2
dist_splits = best_splits.loc['2km']

plt.figure()
plt.plot(data.distance, 500/data.speed)
plt.plot(data.distance, data.heart_rate)
plt.plot(data.distance, data.cadence)

for d, s in dist_splits.split.iteritems():
    ts = s.total_seconds()
    plt.plot([d, d+distance], [ts, ts])
    
plt.ylim(0, 250)

In [ ]:
activities = garmin.get_activities(activityType='rowing', api=api)
activity_data, errors = garmin.load_activities(activities.activityId, api=api)
garmin.activity_data_to_excel(activities, activity_data, api=api)

In [ ]:
activities = garmin.get_activities(activityType='rowing', api=api)
activity_data, errors = garmin.load_activities(activities.activityId, api=api)